In [1]:
# 引入包
import numpy as np
import pandas as pd

In [2]:
# 数据预处理
df = pd.read_csv('application.csv')
print(df.head())

   applicant_id            channel      group     city             event  \
0         10001  web-search-engine    control   Asgard  application_date   
1         10002       social-media    control  Midgard  application_date   
2         10003  web-search-engine  treatment  Midgard  application_date   
3         10004       social-media  treatment   Asgard  application_date   
4         10005       social-media  treatment  Midgard  application_date   

  event_date  
0  2018/10/1  
1  2018/10/1  
2  2018/10/1  
3  2018/10/1  
4  2018/10/1  


In [3]:
# 检查数据的缺失值
# df.info
df.isnull().any()

applicant_id    False
channel         False
group           False
city            False
event           False
event_date      False
dtype: bool

In [4]:
# 检查数据是否有重复值
df.duplicated().any()
# 实际上数据是非常干净的，无需清洗

False

In [5]:
# 数据分组
df_pivot = df.pivot_table(index='applicant_id', columns='event', values='event_date', aggfunc='max')
print(df_pivot)

event        application_date background_check_completed_date  \
applicant_id                                                    
10001               2018/10/1                             NaN   
10002               2018/10/1                      2018/10/17   
10003               2018/10/1                       2018/10/8   
10004               2018/10/1                       2018/10/5   
10005               2018/10/1                       2018/10/7   
...                       ...                             ...   
31694              2018/11/11                             NaN   
31695              2018/11/11                             NaN   
31696              2018/11/11                             NaN   
31697              2018/11/11                             NaN   
31698              2018/11/11                             NaN   

event        background_check_initiated_date card_activation_date  \
applicant_id                                                        
10001           

In [6]:
df_group=df[['applicant_id','group']]
df_group.drop_duplicates(keep='first',inplace=True)
df_channel=df[['applicant_id','channel']]
df_channel.drop_duplicates(keep='first',inplace=True)
df_pivot = pd.merge(df_pivot,df_group,on='applicant_id')
df_pivot = pd.merge(df_pivot,df_channel,on='applicant_id')
print(df_pivot)

       applicant_id application_date background_check_completed_date  \
0             10001        2018/10/1                             NaN   
1             10002        2018/10/1                      2018/10/17   
2             10003        2018/10/1                       2018/10/8   
3             10004        2018/10/1                       2018/10/5   
4             10005        2018/10/1                       2018/10/7   
...             ...              ...                             ...   
21693         31694       2018/11/11                             NaN   
21694         31695       2018/11/11                             NaN   
21695         31696       2018/11/11                             NaN   
21696         31697       2018/11/11                             NaN   
21697         31698       2018/11/11                             NaN   

      background_check_initiated_date card_activation_date card_mailed_date  \
0                                 NaN                  N

<ipython-input-6-1460e4b4ef4c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_group.drop_duplicates(keep='first',inplace=True)
<ipython-input-6-1460e4b4ef4c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_channel.drop_duplicates(keep='first',inplace=True)


In [7]:
# 先筛选出application_date在11月2日之前的applicant
time_data=pd.to_datetime(df_pivot['application_date'],format='%Y/%m/%d')
df_pivot['application_date'] = time_data
e_date = pd.to_datetime('20181102', format='%Y%m%d')
# print(e_date)
df_clear = df_pivot[df_pivot['application_date'] < e_date]
# print(df_clear)
# 其实在这里只需要关心员工是否成功的完成第一单
# 只需要统计出成功的完成第一单的人的申请id，去重，然后添加一个flag，指示是否转换成功就可以了
converted_id = df_clear[(df_clear['first_batch_completed_date'].notna())]['applicant_id'].to_frame()
converted_id.insert(loc=1, column='convertion', value='1')
dedup_df = pd.merge(df_clear, converted_id,how='left',on='applicant_id')
dedup_df['convertion'].fillna(0, inplace=True)
dedup_df['convertion'] = pd.to_numeric(dedup_df['convertion'])
print(dedup_df.info())
# 数据预处理完成

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15584 entries, 0 to 15583
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   applicant_id                     15584 non-null  int64         
 1   application_date                 15584 non-null  datetime64[ns]
 2   background_check_completed_date  13887 non-null  object        
 3   background_check_initiated_date  14081 non-null  object        
 4   card_activation_date             13758 non-null  object        
 5   card_mailed_date                 15584 non-null  object        
 6   first_batch_completed_date       4922 non-null   object        
 7   orientation_completed_date       7214 non-null   object        
 8   group                            15584 non-null  object        
 9   channel                          15584 non-null  object        
 10  convertion                       15584 non-null  int64    

In [8]:
# 计算转化率和方差
conversion_rate = dedup_df.groupby('group')['convertion'].agg([np.mean, np.std])
conversion_rate.columns = ['conversion_rate', 'std_deviation']
print(conversion_rate)

           conversion_rate  std_deviation
group                                    
control           0.263446       0.440523
treatment         0.421307       0.493816


In [9]:
# 假设检验
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

control_converted = dedup_df[dedup_df['group'] == 'control']['convertion']
treatment_converted = dedup_df[dedup_df['group'] == 'treatment']['convertion']
nobs = [control_converted.count(), treatment_converted.count()]
success = [control_converted.sum() ,treatment_converted.sum()]
z_stat, pval = proportions_ztest(success, nobs=nobs)
(lower_con, lower_tre), (upper_con, upper_tre) = proportion_confint(success, nobs=nobs, alpha=0.1)
print(z_stat, pval)
print(lower_con, upper_con)
print(lower_tre, upper_tre)
# 可以看到，转化率均在置信区间内，p值远小于α，z远小于-1.65，故提前背调对于提高转化率是非常有效的

-19.962758464526697 1.1612207557079474e-88
0.25634520730867194 0.2705468403286696
0.4100137225294781 0.43260035326325197


In [10]:
# 验证变化是否cost effective
avg_cost = [(i * 30)/j for (i,j) in zip(nobs, success)]
print(avg_cost)
# 可以看到，转换率提高的同时，转换成功的人均花费减少，是cost eccective的

[113.87531899380241, 71.20697567691602]


In [11]:
#根据channel计算分组转化率
channel_conversion_rate = dedup_df.groupby(['group','channel'])['convertion'].agg(np.mean)
channel_conversion_rate

group      channel               
control    job-search-site           0.155616
           shopper-referral-bonus    0.333804
           social-media              0.313630
           web-search-engine         0.249141
treatment  job-search-site           0.374011
           shopper-referral-bonus    0.490251
           social-media              0.381464
           web-search-engine         0.446154
Name: convertion, dtype: float64